Developing sketch primitive noisifier

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from copy import deepcopy
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

from sketchgraphs.data import flat_array
import sketchgraphs.data as datalib

import img2cad.data_utils as i2c_utils
from img2cad.prerender_images import _render_and_save

In [ ]:
seq_data = flat_array.load_dictionary_flat('../data/sequence_data/sg_t16_validation.npy')
seqs = seq_data['sequences'][:50000]
sketches = [datalib.sketch_from_sequence(seq) for seq in tqdm(seqs)]

In [ ]:
def my_render(sketch):
    _render_and_save(sketch, 'tmp.png', size=64,
        enforce_normalized=False, force_06=True)
    img = io.imread('tmp.png')
    plt.imshow(img)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    _ = [line.set_marker('None') for line in ax.get_xticklines()] 
    _ = [line.set_marker('None') for line in ax.get_yticklines()]
    os.remove('tmp.png')

In [ ]:
num_sketches = 8
num_noisings = 5
num_frames = 20
save_dir = 'noise_imgs'
os.makedirs(save_dir)
if os.path.exists('viable_sketches.npy'):
    viable_sketches = np.load('viable_sketches.npy', allow_pickle=True)
else:
    viable_sketches = [sk for sk in sketches if len(sk.entities) >= 6]
    viable_sketches = np.random.choice(viable_sketches, size=num_sketches)
for frame_idx in tqdm(range(num_frames)):
    plot_idx = 1
    for sketch in viable_sketches:
        i2c_utils.normalize_sketch(sketch)
        ax = plt.subplot(num_sketches, num_noisings+1, plot_idx)
        my_render(sketch)
        if plot_idx == 1:
            ax.set_title('Original', fontsize=8)
        plot_idx += 1
        for _ in range(num_noisings):
            new_sketch = deepcopy(sketch)
            i2c_utils.noisify_sketch_ents(new_sketch)
            i2c_utils.noisify_sketch_global(new_sketch)
            ax = plt.subplot(num_sketches, num_noisings+1, plot_idx)
            plot_idx += 1
            my_render(new_sketch)
    plt.savefig(os.path.join(save_dir, '%02i.png' % frame_idx), dpi=200)
